In [1]:
import os
os.chdir(r'C:\Users\liedt\OneDrive\Georgia Tech\CSE 6242 Data and Visual Analytics\Project\model')
import sqlite3
import pandas as pd
import vtreat
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.ensemble import GradientBoostingRegressor
import skopt
import pickle
from opioid_functions import *

In [ ]:
# define the file
sqlite_file = 'DVADB/DVADB.db'

# open a connection
conn = sqlite3.connect(sqlite_file)

sql = ("""
SELECT npi_summary.npi
    ,npi_summary.nppes_credentials
    ,npi_summary.nppes_provider_gender
    ,npi_summary.nppes_provider_city
    ,npi_summary.nppes_provider_state
    ,npi_summary.nppes_provider_country
    ,npi_summary.specialty_description
    ,npi_summary.total_claim_count
    ,npi_summary.bene_count
    ,npi_summary.opioid_claim_count
    ,npi_summary.opioid_day_supply
    ,npi_summary.opioid_bene_count
    ,npi_summary.opioid_prescriber_rate
    ,npi_summary.average_age_of_beneficiaries
    ,npi_summary.beneficiary_age_less_65_count
    ,npi_summary.beneficiary_age_65_74_count
    ,npi_summary.beneficiary_age_75_84_count
    ,npi_summary.beneficiary_age_greater_84_count
    ,npi_summary.beneficiary_female_count
    ,npi_summary.beneficiary_male_count
    ,npi_summary.beneficiary_race_white_count
    ,npi_summary.beneficiary_race_black_count
    ,npi_summary.beneficiary_race_asian_pi_count
    ,npi_summary.beneficiary_race_hispanic_count
    ,npi_summary.beneficiary_race_nat_ind_count
    ,npi_summary.beneficiary_race_other_count
    ,npi_summary.beneficiary_nondual_count
    ,npi_summary.beneficiary_dual_count
    ,npi_summary.beneficiary_average_risk_score
    ,Physicians_unique.Graduation_year
    ,CASE
        WHEN [graduation_year] BETWEEN '1952' AND '1977' THEN '1952-1977'
        WHEN [graduation_year] BETWEEN '1978' AND '1987' THEN '1978-1987'
        WHEN [graduation_year] BETWEEN '1988' AND '1997' THEN '1988-1997'
        WHEN [graduation_year] BETWEEN '1998' AND '2007' THEN '1998-2007'
        WHEN [graduation_year] BETWEEN '2008' AND '2018' THEN '2008-2018'
        ELSE 'OTHER'
        END AS 'GRAD_YEAR_GROUP'   
    ,Physicians_unique.Number_of_Group_Practice_members
    ,state_reference.state_region
    ,zip_reference.rural_urban_class
    ,zip_reference.zip_land_area_2010
    ,zip_reference.zip_population_density_2010
    ,zip_reference.zip_population_2010
FROM npi_summary
    LEFT JOIN Physicians_unique
        ON npi_summary.npi = Physicians_unique.npi
    LEFT JOIN state_reference
        ON state_reference.nppes_provider_state = npi_summary.nppes_provider_state
    LEFT JOIN zip_reference
        ON zip_reference.zip = npi_summary.nppes_provider_zip5              
WHERE Physicians_unique.NPI is not null
    AND opioid_day_supply <> '0' 
    AND opioid_day_supply <> '';
""")

# read from the main table
df = pd.read_sql_query(
    sql,
    conn
)

df['log_opioid_day_supply'] = np.log(df['opioid_day_supply'])

# replace missing valaues related to zip with mean
df_model['zip_population_2010'].fillna((df_model['zip_population_2010'].mean()), inplace=True)
df_model['zip_land_area_2010'].fillna((df_model['zip_land_area_2010'].mean()), inplace=True)
df_model['zip_population_density_2010'].fillna((df_model['zip_population_density_2010'].mean()), inplace=True)

# take a sample for analysis / modeling
df_model = df.sample(frac=.3)

# quick summary
print(f'dataframe dimensions: {df_model.shape}')
print(f'column names: {df_model.columns}')
print(f' column types: {df_model.dtypes}')

# set index to npi
df_model = df_model.set_index('npi')

# columns that we cannot use for modeling
drop_cols = [
    'opioid_claim_count',
    'opioid_bene_count',
    'opioid_prescriber_rate',
    'opioid_day_supply',
]

df_model.drop(drop_cols, axis=1, inplace=True)

# format columns to numeric
for i in df_model.columns:
    if df_model[i].dtype == object:
        df_model[i] = df_model[i].apply(
            lambda x: 0 if x == '' else x
        )

# clean credentials into usable meta groups
df_model = clean_credentials(df_model)
df_model.drop('nppes_credentials', axis=1, inplace=True)


# list of categorical columns
cols_to_dummy = [
    'nppes_provider_gender', 
    'nppes_provider_city',
    'nppes_provider_state',
    'nppes_provider_country',
    'specialty_description',
    'GRAD_YEAR_GROUP',
    'state_region',
    'rural_urban_class'
]

# dummy out the categorical columns
df_model = dummy_wrapper(df_model, cols_to_dummy)

# variance filter
df_model = variance_threshold(df_model, threshold=.003)

In [ ]:
## EDA 

# distribution of target
plt.style.use(['dark_background'])
plt.rcParams['figure.figsize'] = (14, 10)
sns.kdeplot(df_model['opioid_prescriber_rate'])
plt.show()

# TODO: box plots of categorical to target

# scatterplot risk score to rate
sns.scatterplot(
    'beneficiary_average_risk_score',
    'opioid_prescriber_rate',
    data=df_model.sample(frac=.01)
).set_title('Opioid Prescriber Rate vs. Risk Score')
plt.show()

# scatterplot total claim to rate
sns.scatterplot(
    np.log(df_model['total_claim_count']),
    'opioid_prescriber_rate',
    data=df_model.sample(frac=.01)
).set_title('Opioid Prescriber Rate vs. Total Claim Count')
plt.show()

# scatterplot avg age to rate
sns.scatterplot(
    'average_age_of_beneficiaries',
    'opioid_prescriber_rate',
    data=df_model.sample(frac=.01)
).set_title('Opioid Prescriber Rate vs. Average Beneficiary Age')
plt.show()

# correlation matrix - spearman
corr = pd.DataFrame(df_model.corr(method='spearman'))

# print heatmap of correlation matrix all features
plt.figure(figsize=(14, 10))
sns.heatmap(corr, cbar=False, cmap='YlGnBu').set_title(
    'Correlation Matrix: Full Feature Set'
)
plt.show()

# correlation matrix - key features - spearman
corr_subset = pd.DataFrame(
    df_model.loc[:, 'total_claim_count':'beneficiary_average_risk_score']
    .corr(method='spearman')
)

# print heatmap of correlation matrix select features
plt.figure(figsize=(14, 10))
sns.heatmap(
    corr_subset, cbar=False, cmap='YlGnBu', annot=True
    ).set_title('Correlation Matrix: Select Feature Set')
plt.show()

# distribution of opioid day supply
sns.distplot(df['opioid_day_supply']).set_title('Distribution of Opioid Day Supply - full file')
plt.savefig('opioid_day_supply.png')

# distribution of log opioid day supply
sns.distplot(df['log_opioid_day_supply']).set_title('Distribution of Log Opioid Day Supply - full file')
plt.savefig('log_opioid_day_supply.png')

In [ ]:
## MODEL
# set up train/test 

# divide data into target and predictors
model_target = 'log_opioid_day_supply'
op_target = df_model[model_target].values
op_features = df_model.drop(model_target, axis=1)

# set up test and train
x_train, x_test, y_train, y_test = train_test_split(
    op_features,
    op_target,
    test_size=.3,
    shuffle=True,
    random_state=789651244
)

# initialize the gbm 
op_gbm = GradientBoostingRegressor(
    learning_rate=0.1,
    loss='lad',
    max_depth=200,
    max_features='sqrt',
    min_impurity_decrease=0.0,
    min_impurity_split=None,
    min_samples_leaf=.05,
    min_samples_split=.05,
    n_estimators=200,
    presort='auto',
    random_state=758491629,
    subsample=0.3,
    verbose=10,
    warm_start=False
)

In [ ]:
## MODEL
# set up bayesian grid search
opt = skopt.BayesSearchCV(
    op_gbm,
    {
        "learning_rate": (1e-8, .98, 'log-uniform'),
        "max_depth": (1, 300),
        "n_estimators": (20, 200),
        "max_features": ["sqrt", "log2"],
        "subsample": (0.3, .9, 'log-uniform'),
        "min_samples_split": (.005, .1),
        "min_samples_leaf": (.005, .1)
    },
    n_iter=40,
    cv=5,
    scoring='neg_mean_absolute_error'
)

# fit parameters
op_opt_gbm = opt.fit(x_train, y_train)

# cross validate the bayesian optimized gbm model
cross_validate(
    op_opt_gbm,
    X=x_train,
    y=y_train,
    cv=10
)

# show the best params
print("val. score: %s" % opt.best_score_)
print(opt.best_estimator_)

In [ ]:
## MODEL
# initialize the gbm regressor
op_gbm_bayes = GradientBoostingRegressor(
    alpha=0.9, criterion='friedman_mse', init=None,
    learning_rate=0.15066211175185815, loss='lad', max_depth=203,
    max_features='log2', max_leaf_nodes=None,
    min_impurity_decrease=0.0, min_impurity_split=None,
    min_samples_leaf=0.005, min_samples_split=0.005,
    min_weight_fraction_leaf=0.0, n_estimators=200,
    presort='auto', random_state=758491629, subsample=0.9,
    verbose=10, warm_start=False
)

# fit the regressor
op_gbm_bayes_fit = op_gbm_bayes.fit(x_train, y_train)

# cross validate the gbm model
cross_validate(
    op_gbm_bayes_fit,
    X=x_train,
    y=y_train,
    cv=10,
    scoring='neg_mean_absolute_error'
)

# extract variable importance for the gbm model
df_vimp, vimp_columns = extract_vimp(
    op_gbm_bayes_fit,
    column_names=x_train,
    threshold=.004
)

In [ ]:
# plot variable importance
plt.rcParams['figure.figsize'] = (14, 10)
sns.barplot(
    x='tree_vimp',
    y='variable',
    data=df_vimp
).set_title('Variable Importance: Bayesian Gradient Boosting')

plt.show()

# get model train predictions
y_pred = op_gbm_bayes_fit.predict(x_train)
print(f'train predictions mean {np.mean(y_pred)}')
print(f'train predictions median {np.median(y_pred)}')
print(f'train predictions var {np.var(y_pred)}')

# plot predictions distribution
sns.distplot(y_pred).set_title('Train Predictions Distribution')
plt.show()

# plot predictions vs. actuals - train set
sns.scatterplot(
    x=y_pred,
    y=y_train
).set_title('Opioid Rate Predictions vs. Train')
plt.xlabel('predictions')
plt.ylabel('train actuals')
plt.show()

# put train predictions and actuals together
df_comb = pd.concat([pd.Series(y_pred), pd.Series(y_train)], axis=1)
df_comb.columns = ['predictions', 'actuals']

# get difference
df_comb['error'] = np.abs(df_comb['predictions'] - df_comb['actuals'])

# plot predictions vs. actuals - train set
sns.scatterplot(
    x='predictions',
    y='actuals',
    data=df_comb.sample(frac=.3),
    hue='error'
).set_title('Opioid Rate Predictions vs. Train')
plt.xlabel('predictions')
plt.ylabel('train actuals')
plt.show()

# get model test predictions
y_pred_test = op_gbm_bayes_fit.predict(x_test)
print(f'test predictions mean {np.mean(y_pred_test)}')
print(f'test predictions median {np.median(y_pred_test)}')
print(f'test predictions var {np.var(y_pred_test)}')

# plot predictions distribution
sns.distplot(y_pred_test).set_title('Test Predictions Distribution')
plt.show()

# plot predictions vs. actuals - train set
sns.scatterplot(
    x=y_pred_test,
    y=y_test
).set_title('Opioid Rate Predictions vs. Test')
plt.xlabel("Predictions")
plt.ylabel("Actuals")
plt.show()

# put train predictions and actuals together
df_comb = pd.concat([pd.Series(y_pred_test), pd.Series(y_test)], axis=1)
df_comb.columns = ['predictions', 'actuals']

# get difference
df_comb['error'] = np.abs(df_comb['predictions'] - df_comb['actuals'])

# plot predictions vs. actuals - train set
sns.scatterplot(
    x='predictions',
    y='actuals',
    data=df_comb.sample(frac=.3),
    hue='error',
    size='error'
).set_title('Opioid Rate Predictions vs. Test')
plt.xlabel('predictions')
plt.ylabel('train actuals')
plt.show()

In [ ]:
# TODO: shap values
# TODO: transformation functions for predicting on entire data set
# TODO: output frames for plotting shap, residuals
# TODO: output final model for predicting on entire dataset

# final model to production
pickle.dump(
    op_gbm_bayes_fit,
    open('opioid_gbm_full.sav', 'wb')
)

# final features to production
pickle.dump(
    x_train.columns,
    open('opioid_features.sav', 'wb')
)